In [1]:
import pyspark.sql
from pyspark.sql import functions as F
import pandas as pd

# 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/19 16:02:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
pd_cases = pd.read_csv('311.csv')
pd_dept = pd.read_csv('dept.csv')

In [20]:
cases = spark.createDataFrame(pd_cases)

In [19]:
dept = spark.createDataFrame(pd_dept)

In [8]:
cases.describe().show(vertical=True)

23/05/19 16:05:20 WARN TaskSetManager: Stage 3 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 summary              | count                
 case_id              | 100000               
 case_opened_date     | 100000               
 case_closed_date     | 100000               
 SLA_due_date         | 100000               
 case_late            | 100000               
 num_days_late        | 100000               
 case_closed          | 100000               
 dept_division        | 100000               
 service_request_type | 100000               
 SLA_days             | 100000               
 case_status          | 100000               
 source_id            | 100000               
 request_address      | 100000               
 council_district     | 100000               
-RECORD 1------------------------------------
 summary              | mean                 
 case_id              | 1.01422656004933E9   
 case_opened_date     | null                 
 case_closed_date     | NaN                  
 SLA_due_date         | NaN       

In [9]:
dept.describe().show(vertical=True)

-RECORD 0--------------------------------------
 summary                | count                
 dept_division          | 39                   
 dept_name              | 39                   
 standardized_dept_name | 39                   
 dept_subject_to_SLA    | 39                   
-RECORD 1--------------------------------------
 summary                | mean                 
 dept_division          | null                 
 dept_name              | NaN                  
 standardized_dept_name | null                 
 dept_subject_to_SLA    | null                 
-RECORD 2--------------------------------------
 summary                | stddev               
 dept_division          | null                 
 dept_name              | null                 
 standardized_dept_name | null                 
 dept_subject_to_SLA    | null                 
-RECORD 3--------------------------------------
 summary                | min                  
 dept_division          | 311 Call Cente

## 2. Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

Im going to do this with dept instead

In [10]:
dept.write.csv('data/dept', mode='overwrite', header=True)

In [12]:
dept.write.json('data/dept', mode='overwrite')

### Inspect your folder structure. What do you notice?
The files are split up into smaller files

## 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [13]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [22]:
cases.select('case_opened_date').show(2)

+----------------+
|case_opened_date|
+----------------+
|     1/1/18 0:42|
|     1/1/18 0:46|
+----------------+
only showing top 2 rows



23/05/19 16:17:54 WARN TaskSetManager: Stage 14 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [24]:
fmt = 'M/d/yy H:m'

In [29]:
cases = cases.withColumn(
    'case_opened_date',
    F.to_timestamp('case_opened_date', fmt)
    ).\
    withColumn(
    'case_closed_date',
    F.to_timestamp('case_closed_date', fmt)
    ).\
    withColumn(
    'SLA_due_date',
    F.to_timestamp('SLA_due_date', fmt)
    )

In [30]:
cases.select('case_opened_date').show(2)

+-------------------+
|   case_opened_date|
+-------------------+
|2018-01-01 00:42:00|
|2018-01-01 00:46:00|
+-------------------+
only showing top 2 rows



23/05/19 16:24:25 WARN TaskSetManager: Stage 16 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [31]:
cases.select('SLA_due_date').show(3)

+-------------------+
|       SLA_due_date|
+-------------------+
|2020-09-26 00:42:00|
|2018-01-05 08:30:00|
|2018-01-05 08:30:00|
+-------------------+
only showing top 3 rows



23/05/19 16:24:28 WARN TaskSetManager: Stage 17 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [32]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [34]:
cases.select('case_late', 'case_closed', 'case_status').show(3)

+---------+-----------+-----------+
|case_late|case_closed|case_status|
+---------+-----------+-----------+
|       NO|        YES|     Closed|
|       NO|        YES|     Closed|
|       NO|        YES|     Closed|
+---------+-----------+-----------+
only showing top 3 rows



23/05/19 16:25:42 WARN TaskSetManager: Stage 18 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [36]:
cases.select('case_status').distinct().show()

23/05/19 16:26:37 WARN TaskSetManager: Stage 19 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+
|case_status|
+-----------+
|       Open|
|     Closed|
+-----------+



case_status is redundant with case_closed

In [37]:
cases = cases.withColumn(
    'case_closed', 
    F.expr('case_closed') == 'YES').\
    withColumn(
    'case_late', 
    F.expr('case_late') == 'YES')

In [40]:
cases = cases.drop('case_status')

In [41]:
cases.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



23/05/19 16:29:54 WARN TaskSetManager: Stage 24 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [44]:
cases = cases.withColumn(
    'request_address',
    F.trim(F.lower('request_address'))
)

In [45]:
cases.show(1, vertical=True)

23/05/19 16:34:19 WARN TaskSetManager: Stage 26 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
only showing top 1 row



# part 2
## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [65]:
cases.filter('case_closed == False').\
    withColumn('newest_open',
        F.datediff(F.current_timestamp(), F.max('case_opened_date'))
    ).show()

AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [case_id#2284L, case_opened_date#2375, case_closed_date#2390, SLA_due_date#2405, case_late#2473, num_days_late#2289, case_closed#2458, dept_division#2291, service_request_type#2292, SLA_days#2293, source_id#2295, request_address#2749, council_district#2297L, datediff(cast(current_timestamp() as date), cast(max(case_opened_date#2375) as date)) AS newest_open#2922]
+- Filter (case_closed#2458 = false)
   +- Project [case_id#2284L, case_opened_date#2375, case_closed_date#2390, SLA_due_date#2405, case_late#2473, num_days_late#2289, case_closed#2458, dept_division#2291, service_request_type#2292, SLA_days#2293, source_id#2295, trim(lower(request_address#2296), None) AS request_address#2749, council_district#2297L]
      +- Project [case_id#2284L, case_opened_date#2375, case_closed_date#2390, SLA_due_date#2405, case_late#2473, nu